In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/climate-text-dataset/Master file_10k papers.xlsx
/kaggle/input/climate-text-dataset/Human labelled_DTU.xlsx


In [4]:
df=pd.read_excel("/kaggle/input/climate-text-dataset/Human labelled_DTU.xlsx",header=1)
# clean up
df = df.dropna(how="all").dropna(axis=1, how="all")
df = df.loc[:, ~df.columns.astype(str).str.startswith("Unnamed")]
df

/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Coder name,Article ID,Paper_Author/s,Paper title,Year of publication,DOI,URL,Abstracts,Accept/Reject,"If Accept, identify theme"
0,Shreya,Scopus_4859,"A.K., Menzies, Allyson K.; E., Bowlesa, E.; M....","""I see my culture starting to disappear"": Anis...",2022,10.1139/facets-2021-0066,NaN,Climate change disproportionately affects Indi...,Accept,"Empowering People & Governance, Policy"
1,Shreya,OA_0102,Claudia Quitmann|Rainer Sauerborn|Ina Danquah|...,"‘Climate change mitigation is a hot topic, but...",2022,NaN,https://doi.org/10.1136/medethics-2021-107971,Physical and mental well-being are threatened ...,Reject,NaN
2,Shreya,Scopus_4071,"R., Losciale, Riccardo; R., Hay, Rachel; M.A.,...","‘The public perception of the role, importance...",2022,10.1016/j.envdev.2022.100757,NaN,"Seagrass meadows, one of the world′s greatest ...",Accept,Ecosystem & Non-human Well-being
3,Shreya,OA_0799,Lucy H. Bird|Emma F. Thomas|Michael Wenzel,‘We despair’: Examining the role of political ...,2024,NaN,https://doi.org/10.1002/ejsp.3052,"Abstract Anecdotally, people often report feel...",Reject,NaN
4,Shreya,Scopus_4993,"J.G., Rosen, Joseph Gregory; D.M., Mulenga, Dr...",“Burnt by the scorching sun”: climate-induced ...,2021,10.1186/s12889-021-11560-8,NaN,Background: Climate-induced disruptions like d...,Reject,NaN
...,...,...,...,...,...,...,...,...,...,...
1730,Philip,WoS_5887,"Bezáková, M; Bezák, P",Which sustainability objectives are difficult ...,2022,10.1016/j.landusepol.2022.106020,http://dx.doi.org/10.1016/j.landusepol.2022.10...,The Poloniny National Park (Poloniny) is a for...,Reject,NaN
1731,Philip,WoS_6145,"Gunn, KM; Turnbull, DA; Dollman, J; Kettler, L...",Why are some drought-affected farmers less dis...,2021,10.1111/ajr.12695,http://dx.doi.org/10.1111/ajr.12695,Objective To identify the modifiable psycholog...,Reject,NaN
1732,Philip,WoS_5711,"Aithal, S; Aithal, V; Kei, J; Wilson, M",Wideband Tympanometry Findings in Healthy Neon...,2022,10.1055/a-1925-7830,http://dx.doi.org/10.1055/a-1925-7830,Objectives The objective of the present study ...,Reject,NaN
1733,Philip,WoS_0343,"Jussila, J; Nagy, E; Lähtinen, K; Hurmekoski, ...",Wooden multi-storey construction market develo...,2022,10.14214/sf.10609,http://dx.doi.org/10.14214/sf.10609,Climate change sets high pressures on the cons...,Reject,NaN


In [7]:
# df = your dataframe

# 1) Keep only rows where Accept/Reject == "Accept" (case-insensitive)
accept_df = df[df["Accept/Reject"].astype(str).str.strip().str.lower() == "accept"].copy()

# 2) Clean the theme column and count unique themes (drop blanks/NaN)
theme_col = "If Accept, identify theme"

themes = (
    accept_df[theme_col]
    .astype(str)
    .str.strip()
    .replace({"": pd.NA, "nan": pd.NA, "none": pd.NA})
    .dropna()
)

n_unique = themes.nunique()
unique_list = sorted(themes.unique())

print("Accepted rows:", len(accept_df))
print("Unique themes (Accept only):", n_unique)
print("Themes list:", unique_list)

Accepted rows: 199
Unique themes (Accept only): 12
Themes list: ['Access to Services and Wellbeing', 'Better Well-being Metric', 'Climate change mitigation options & Well-being', 'Ecosystem & Non-human Well-being', 'Empowering People & Governance, Policy', 'Equity, Social Justice, Just Transition', 'Implemented mitigation Options and Well-being', 'Integrated Responses and Well-being', 'SDGs and Well-being', 'Systems Thinking- Modelling and Supply Chain', 'Well-being Loss from Climate Mitigation', 'Well-being, Community & Societal Systems Thinking']


In [8]:
theme_counts = themes.value_counts().reset_index()
theme_counts.columns = ["Theme", "Count"]
theme_counts["Percent"] = (theme_counts["Count"] / theme_counts["Count"].sum() * 100).round(2)

display(theme_counts)

,Theme,Count,Percent
0,Climate change mitigation options & Well-being,84,42.42
1,"Empowering People & Governance, Policy",31,15.66
2,Implemented mitigation Options and Well-being,21,10.61
3,"Well-being, Community & Societal Systems Thinking",11,5.56
4,SDGs and Well-being,10,5.05
5,"Equity, Social Justice, Just Transition",8,4.04
6,Better Well-being Metric,7,3.54
7,Ecosystem & Non-human Well-being,6,3.03
8,Integrated Responses and Well-being,6,3.03
9,Access to Services and Wellbeing,6,3.03


In [9]:
prediction=pd.read_csv("/kaggle/input/climate-text-dataset/hierarchical_predictions_1.csv")
prediction

,ID_New,Article Title,Prediction_Accept_Reject,Predicted_Theme,Binary_Confidence
0,OA_3712,NaN,Accept,Climate change mitigation options & Well-being,0.753731
1,WoS_1385,"It ' s one thing after another, after another...",Reject,NaN,0.000165
2,Scopus_5109,"""A Return to and of the Land"": Indigenous Know...",Accept,"Well-being, Community & Societal Systems Thinking",0.568593
3,Scopus_4859,"""I see my culture starting to disappear"": Anis...",Accept,Integrated Responses and Well-being,0.999564
4,Scopus_1176,"""Impact of Climate Change on Coastal Cities: A...",Reject,NaN,0.001463
...,...,...,...,...,...
10170,OA_4235,ҚАЗАҚСТАН РЕСПУБЛИКАСЫНДАҒЫ АЗЫҚ-ТҮЛІК ЖҮЙЕСІН...,Reject,NaN,0.000369
10171,OA_3078,СУЧАСНИЙ СТАН БАНКІВСЬКОЇ СИСТЕМИ РЕСПУБЛІКИ К...,Reject,NaN,0.000311
10172,OA_4940,Պարենային անվտանգության ոլորտում ԵԱՏՄ երկրների...,Reject,NaN,0.000248
10173,OA_4085,آثار تغير المناخ على الاقتصاد الأزرق في مصر,Accept,Climate change mitigation options & Well-being,0.799474


In [10]:
# df = your dataframe

# 1) Keep only rows where Accept/Reject == "Accept" (case-insensitive)
accept_df = prediction[prediction["Prediction_Accept_Reject"].astype(str).str.strip().str.lower() == "accept"].copy()

# 2) Clean the theme column and count unique themes (drop blanks/NaN)
theme_col = "Predicted_Theme"

themes = (
    accept_df[theme_col]
    .astype(str)
    .str.strip()
    .replace({"": pd.NA, "nan": pd.NA, "none": pd.NA})
    .dropna()
)

n_unique = themes.nunique()
unique_list = sorted(themes.unique())

print("Accepted rows:", len(accept_df))
print("Unique themes (Accept only):", n_unique)
print("Themes list:", unique_list)

Accepted rows: 1663
Unique themes (Accept only): 11
Themes list: ['Access to Services and Wellbeing', 'Better Well-being Metric', 'Climate change mitigation options & Well-being', 'Ecosystem & Non-human Well-being', 'Equity, Social Justice, Just Transition', 'Implemented mitigation Options and Well-being', 'Integrated Responses and Well-being', 'SDGs and Well-being', 'Systems Thinking- Modelling and Supply Chain', 'Well-being Loss from Climate Mitigation', 'Well-being, Community & Societal Systems Thinking']


In [11]:
theme_counts = themes.value_counts().reset_index()
theme_counts.columns = ["Theme", "Count"]
theme_counts["Percent"] = (theme_counts["Count"] / theme_counts["Count"].sum() * 100).round(2)

display(theme_counts)

,Theme,Count,Percent
0,Climate change mitigation options & Well-being,719,43.24
1,"Well-being, Community & Societal Systems Thinking",184,11.06
2,Better Well-being Metric,180,10.82
3,Integrated Responses and Well-being,159,9.56
4,SDGs and Well-being,148,8.90
5,Implemented mitigation Options and Well-being,124,7.46
6,"Equity, Social Justice, Just Transition",61,3.67
7,Well-being Loss from Climate Mitigation,32,1.92
8,Access to Services and Wellbeing,30,1.80
9,Systems Thinking- Modelling and Supply Chain,16,0.96
